In [ ]:
# importing all necessary modules and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import imblearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from scipy.stats import randint
import tldextract as tld
from tldextract import tldextract
import re
from urllib.parse import urlparse
import requests
from time import time
import time
from ipwhois import IPWhois
import whois
import socket
from datetime import datetime
from collections.abc import Iterable
import dns.resolver
import os


In [63]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(BASE_DIR, "data", "selected_features.csv")
extracted_df=pd.read_csv(file_path)
extracted_df

,qty_slash_url,length_url,qty_dot_domain,qty_vowels_domain,domain_length,qty_dot_directory,qty_underline_directory,qty_slash_directory,qty_equal_directory,qty_at_directory,...,qty_asterisk_file,file_length,time_response,asn_ip,time_domain_activation,time_domain_expiration,qty_nameservers,qty_mx_servers,ttl_hostname,phishing
0,0.022727,0.005047,0.095238,0.065574,0.057269,0.10,0.055556,0.086957,0.166667,0.022727,...,0.016393,0.006488,0.030641,0.153585,0.000000,0.000000,0.10,0.00,0.001477,1
1,0.068182,0.052632,0.095238,0.081967,0.052863,0.20,0.055556,0.173913,0.166667,0.022727,...,0.016393,0.008110,0.038058,0.091029,0.032628,0.006689,0.10,0.05,0.015775,1
2,0.022727,0.002644,0.095238,0.049180,0.044053,0.05,0.055556,0.086957,0.166667,0.022727,...,0.016393,0.000811,0.049132,0.012045,0.000000,0.000000,0.10,0.15,0.000976,0
3,0.113636,0.018505,0.095238,0.114754,0.066079,0.15,0.166667,0.260870,0.166667,0.022727,...,0.016393,0.008110,0.035785,0.051686,0.000000,0.000000,0.10,0.00,0.000484,1
4,0.000000,0.003605,0.095238,0.081967,0.066079,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035804,0.136024,0.393733,0.013599,0.10,0.05,0.005949,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88642,0.000000,0.004566,0.142857,0.081967,0.083700,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.028810,0.021632,0.309968,0.014839,0.15,0.05,0.005949,0
88643,0.000000,0.007210,0.095238,0.229508,0.132159,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035480,0.066953,0.283922,0.019136,0.10,0.10,0.000979,0
88644,0.113636,0.015862,0.047619,0.098361,0.079295,0.10,0.055556,0.260870,0.166667,0.022727,...,0.016393,0.009732,0.044287,0.997324,0.103792,0.031584,0.10,0.25,0.023796,1
88645,0.022727,0.005768,0.095238,0.081967,0.101322,0.05,0.055556,0.086957,0.166667,0.022727,...,0.016393,0.000811,0.033172,0.120236,0.000000,0.000000,0.05,0.05,0.000088,1


In [64]:
new_x=extracted_df.drop("phishing",axis=1)
new_y=extracted_df["phishing"]

In [65]:
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(new_x, new_y)

In [66]:
new_x_train,new_x_test,new_y_train,new_y_test=train_test_split(x_resampled,y_resampled,test_size=0.2,random_state=42)

In [67]:
#Random Forest
model_1=RandomForestClassifier()
model_1.fit(new_x_train,new_y_train)
model_1.score(new_x_test,new_y_test)

0.9755172413793104

In [68]:
y_preds=model_1.predict(new_x_test)
y_preds

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [69]:
print(classification_report(new_y_test,y_preds))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     11716
           1       0.97      0.98      0.98     11484

    accuracy                           0.98     23200
   macro avg       0.98      0.98      0.98     23200
weighted avg       0.98      0.98      0.98     23200



In [70]:
confusion_matrix(new_y_test,y_preds)

array([[11381,   335],
       [  233, 11251]], dtype=int64)

In [71]:
model_1.score(new_x_train,new_y_train)

0.9999245689655173

In [72]:
accuracy=accuracy_score(new_y_test,y_preds)
accuracy

0.9755172413793104

In [74]:

def find_features(url):
    input_features = []

    # URL features
    url_slashes = url.count("/")
    url_length = len(url)
    extracted_url = tldextract.extract(url)
    domain = extracted_url.domain
    domain_dots = domain.count(".")
    vowels = len(re.findall(r"[aeiouAEIOU]", domain))
    domain_length = len(domain)

    input_features.extend([url_slashes, url_length, domain_dots, vowels, domain_length])

    # Directory Features
    parsed_url = urlparse(url)
    directory = "/".join(parsed_url.path.strip("/").split("/")[:-1])
    directory = "/" + directory

    path_symbols_list = [".", "_", "/", "=", "@", "&", "!", "~", ",", "*", "$", "%"]
    input_features.extend([directory.count(symbol) for symbol in path_symbols_list])

    input_features.append(len(directory))

    # File-based features
    file_symbol_list = [".", "/", "?", "=", "@", "%20", ",", "+", "*"]
    file_name = parsed_url.path.split("/")[-1]
    input_features.extend([file_name.count(symbol) for symbol in file_symbol_list])

    input_features.append(len(file_name))

    # Response time features
    try:
        start_time = time.time()
        response = requests.get(url, timeout=5)
        end_time = time.time()
        response_time = end_time - start_time
    except requests.exceptions.RequestException:
        response_time = -1

    input_features.append(response_time)

    # Network and domain features
    try:
        ip_address = socket.gethostbyname(parsed_url.netloc)
        asn = IPWhois(ip_address).lookup_whois().get("asn", 0)
    except Exception:
        asn = 0  # Default ASN

    input_features.append(int(asn))

    try:
        domain_name = whois.whois(url)
        start_date = domain_name.creation_date
        end_date = domain_name.expiration_date

        if isinstance(start_date, Iterable):
            start_date = start_date[0]
        if isinstance(end_date, Iterable):
            end_date = end_date[0]

        today = datetime.today()
        creation_date = (today - start_date).days if start_date else -1
        expiration_date = (end_date - today).days if end_date else -1
    except Exception:
        creation_date, expiration_date = -1, -1

    input_features.extend([creation_date, expiration_date])

    try:
        nameservers = len(domain_name.name_servers)
    except Exception:
        nameservers = 0

    input_features.append(nameservers)

    try:
        mx_records = dns.resolver.resolve(parsed_url.netloc.replace("www.", ""), "MX")
        mx_count = len(mx_records)
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        mx_count = 0

    input_features.append(mx_count)

    try:
        ttl_records = dns.resolver.resolve(parsed_url.netloc, "A")
        ttl_value = ttl_records.rrset.ttl
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        ttl_value = -1

    input_features.append(ttl_value)

    return input_features


In [95]:

url=input("Enter the URL \n")
input_features=find_features(url)
print(input_features)


[8, 124, 0, 6, 13, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.6523840427398682, 13335, 0, 364, 2, 0, 295]


In [82]:
feature_names=['qty_slash_url', 'length_url', 'qty_dot_domain', 'qty_vowels_domain',
       'domain_length', 'qty_dot_directory', 'qty_underline_directory',
       'qty_slash_directory', 'qty_equal_directory', 'qty_at_directory',
       'qty_and_directory', 'qty_exclamation_directory', 'qty_tilde_directory',
       'qty_comma_directory', 'qty_asterisk_directory', 'qty_dollar_directory',
       'qty_percent_directory', 'directory_length', 'qty_dot_file',
       'qty_slash_file', 'qty_questionmark_file', 'qty_equal_file',
       'qty_at_file', 'qty_space_file', 'qty_comma_file', 'qty_plus_file',
       'qty_asterisk_file', 'file_length', 'time_response', 'asn_ip',
       'time_domain_activation', 'time_domain_expiration', 'qty_nameservers',
       'qty_mx_servers', 'ttl_hostname']

In [96]:
input_features = np.array(input_features).reshape(1, -1)# Shape: (1, n_features)
preds=model_1.predict(input_features)
preds

c:\Users\bhuva\miniconda3\envs\env\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [94]:
input_features = np.array(input_features).reshape(1, -1)# Shape: (1, n_features)
features_df=pd.DataFrame(input_features,columns=feature_names)
preds=model_1.predict(features_df)
if(preds[0]==0):
    print("Don't worry you are safe, its a Secure link")
else:
    print("Beware! its a trap")

Don't worry you are safe, its a Secure link


In [45]:
# import joblib

# # Assuming `model` is your trained model
# joblib.dump(model_1, 'model.pkl')  
